In [1]:
import json
from web3 import Web3
import os
import platform
from getpass import getpass

In [2]:
operating_system = platform.system()
d = '/'
if operating_system == 'Windows':
    d = '\\'

ganache_url = "HTTP://127.0.0.1:7545"
web3 = Web3(Web3.HTTPProvider(ganache_url))

# This is our main account that controls everything
web3.eth.defaultAccount = web3.eth.accounts[0]

In [3]:
web3.eth.accounts

['0xb6ef75b96D856a6cE0C796be4a7ABfB81E5D600A',
 '0xa2D0B6dbCd04a789da44CCECA9d66B6dfc93A124',
 '0xEE4F5078d802207d68eE9668AE6b82FCFCF7e290',
 '0x9cbFF84198C1c3ed898C52Fbc0076Bf008476C4B',
 '0x18a31E094820e4C9C79a073A0927FA72635b18C7',
 '0xDED00A7f19D445B2c6D3ddFffbf78BB490771937',
 '0x37594b8C8aC1D94457C86e30dE793099CE659817',
 '0x6a1ab8E6bb7e9CC3463997FC7F38e3B3855282b5',
 '0xAfd3bB14ff7735B0D9792e5bcc9BB5e758a32ccB',
 '0x8b23fa43919cB11b9CDa04FA45B92aaA0A0Adcd8']

In [4]:
def getContract(filename):
    with open('..'+ d +'build'+ d +'contracts'+ d +filename) as contractFile:
        contractJson=json.load(contractFile)
        contractABI =contractJson['abi']
        contractAdd=web3.toChecksumAddress(contractJson['networks']['5777']['address']) 
        return contractABI, contractAdd



# Get the organization contract address
organizationContractABI, organizationContractAdd = getContract('Organization.json')	
organizationContract = web3.eth.contract(address=organizationContractAdd, abi=organizationContractABI)

#get the user contract address
userContractABI,userContractAdd = getContract('User.json')
userContract = web3.eth.contract(address=userContractAdd, abi=userContractABI)

# Get the accounts contract address
accountsContractABI,accountsContractAdd = getContract('Accounts.json')
accountsConract = web3.eth.contract(address=accountsContractAdd, abi=accountsContractABI)

loansContractABI, loansContractAddress= getContract('Loans.json')


In [49]:
# This should be changed to be a more robus way. We can give the loansContractAddress using the constructor
# but this has problems when doing this in the 2_deploy_contracts.js
userContract.functions.setLoansContractAddress(loansContractAddress).transact()
organizationContract.functions.setLoansContractAddress(loansContractAddress).transact()

HexBytes('0xf076f0a97415c3912a5f1ca81f8b173fe5948149c1815b3b85659b5e948f91da')

HexBytes('0x4c1a99eeeb76eb40f1a85e3523744be48fa3c103f033ebc0b339a878aba16a54')

In [38]:
accountsConract.functions.add(web3.eth.accounts[1], False).transact()
accountsConract.functions.add(web3.eth.accounts[2], False).transact()
accountsConract.functions.add(web3.eth.accounts[5], True).transact()
accountsConract.functions.add(web3.eth.accounts[6], True).transact()

HexBytes('0xccbce479f7a7e16ee9bd8f200d60aae1b9b0a9af17dbc22103f424bb15e14026')

HexBytes('0xb66182df68fb9649f8da41ee7b3f04aff9aae3f7e0ca8f9ac53f2d9d81a7ee8f')

HexBytes('0x4ee92c0bccb10c4326dfbfb86c5b1a089b55b02b3a4bd3d55ffc68dfdf40c2f2')

HexBytes('0x23d413473c214d710d0393c62a3e60f6a2649836fb6ff7c5bd3ea8b4846492bd')

In [39]:
def deleteUser():
    accountsConract.functions.deleteAccount(web3.eth.accounts[1]).transact()

def createLoan(_address,_loaner,_amount,_loansContractAddress):
    check=organizationContract.functions.createLoan(_address,_loaner,_amount).transact()	
    return check

In [49]:
deleteUser()

In [18]:
def getPendingLoans(_userContract, _loanieAddress, _pk=None):
    pendingLoans = _userContract.functions.getPendingLoans().buildTransaction({'gas': 70000,'gasPrice': web3.toWei('1', 'gwei'),'from': _loanieAddress,'nonce': web3.eth.getTransactionCount(_loanieAddress)}) 
    private_key = getpass('Enter your key: ') if _pk is not None else _pk
    test_transaction = {'gas': 70000,'gasPrice': web3.toWei('1', 'gwei'),'from': _loanieAddress,
                        'nonce': web3.eth.getTransactionCount(_loanieAddress), 'data':pendingLoans['data']}
    signed_txn = web3.eth.account.signTransaction(pendingLoans, private_key=private_key)
    print(signed_txn)
    transaction_hash = web3.eth.sendRawTransaction(signed_txn.rawTransaction)
    print('Transaction Block: \n', web3.eth.getTransaction(transaction_hash))
    print('Transaction Receipt: \n', web3.eth.getTransactionReceipt(transaction_hash))
    print('Pending loan ended!')

In [45]:
def getPendingLoans2(_userContract, _loanieAddress):
    transaction = _userContract.functions.getPendingLoans(
    ).buildTransaction({
    'gas': 70000,
    'gasPrice': web3.toWei('1', 'gwei'),
    'from': _loanieAddress,
    'nonce': web3.eth.getTransactionCount(_loanieAddress)
    }) 
    private_key = getpass('Enter your key: ')
    signed_txn = web3.eth.account.signTransaction(transaction, private_key=private_key)
    print('Raw Transaction: ', signed_txn.rawTransaction)
    print('Hex Raw Transaction: ', web3.toHex(signed_txn.rawTransaction))
    transaction_hash = web3.eth.sendRawTransaction(web3.toHex(signed_txn.rawTransaction))
    print(web3.toHex(transaction_hash))

In [ ]:
userContract.functions.getPendingLoans({'from': web3.eth.accounts[0], 'gas':1000000, 'value': web3.toWei(0.01, "ether")})

In [41]:
# Suppose bank wants to create a loan
loanieAddress = input("Enter the addrses: ")

### Check to see whether it is a user or organization address
index = accountsConract.functions.getIndex(loanieAddress).call()

Enter the addrses: 0xEE4F5078d802207d68eE9668AE6b82FCFCF7e290


In [42]:
index

1

In [10]:
# Second index account private key
pk = '8700bbbe5cc527282fc13aa85dfc9cbe2493cdaa4b87fea14c0b30ad56c129d7'

In [46]:
if index != -1:
    loanieType = accountsConract.functions.getType(index).call()
    if not loanieType:
        #res = organizationContract.functions.createLoan(loanieAddress).call()
        res=True
        print(res)
        choice=input("pending loans Y or N")
        if choice == "Y":
            #getPendingLoans(userContract, loanieAddress, pk)
            getPendingLoans2(userContract, loanieAddress)
    else :
        choice=input("create loan Y/N?")
        if choice=="Y":
            if createLoan(web3.eth.accounts[2],web3.eth.accounts[6],1000,loansContractAddress):
                print("Loan created")
else:
    print("This account is not registered in our system.")

True
pending loans Y or NY
Enter your key: ········
Raw Transaction:  b'\xf8j\x1e\x84;\x9a\xca\x00\x83\x01\x11p\x94\x80\x03\xbd\x91j\x03\xfb\x0bb\xc5PevC\xa8\x0ea_,~\x80\x84\xee$\xafZ\x82\n\x96\xa0=\x93\xcf\xcf\x0cm\xf7\xa4\xc4\x02\x01\xcc\x91\x9e@A\x89\xf7\xb94\x8b\xa5\xa0\x82\x84\t\xa9\x08/\xd2\xd2$\xa0\t\x99L\xee\xedY\x05\xecB\x1d%\x05\xdd\x95iq\x1f\xe3\x90w\x8b\x88\xd8\xb3\xd2tR\xec[v\x9d\x11'
Hex Raw Transaction:  0xf86a1e843b9aca0083011170948003bd916a03fb0b62c550657643a80e615f2c7e8084ee24af5a820a96a03d93cfcf0c6df7a4c40201cc919e404189f7b9348ba5a0828409a9082fd2d224a009994ceeed5905ec421d2505dd9569711fe390778b88d8b3d27452ec5b769d11
0xbd03f823229008f92792f07a85c8887cba3a86a776a7a9c6504d134dd776eca8
